In [1]:
import torchvision
import torch.nn as nn

from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

In [2]:
model_backbone = torchvision.models.vgg16_bn(weights='DEFAULT')

backbone = model_backbone.features

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to C:\Users\Admin/.cache\torch\hub\checkpoints\vgg16_bn-6c64b313.pth
100%|██████████| 528M/528M [00:28<00:00, 19.6MB/s] 


In [3]:
print(backbone)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU(inplace=True)
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (9): ReLU(inplace=True)
  (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): ReLU(inplace=True)
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 

In [ ]:
from gcn_lib import Grapher, act_layer
import torch.nn as nn
from torch.nn import Sequential
model_backbone = torchvision.models.vgg16_bn(weights='DEFAULT')

backbone = model_backbone.features
class FFN(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act='relu', drop_path=0.0):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Sequential(
            nn.Conv2d(in_features, hidden_features, 1, stride=1, padding=0),
            nn.BatchNorm2d(hidden_features),
        )
        self.act = act_layer(act)
        self.fc2 = nn.Sequential(
            nn.Conv2d(hidden_features, out_features, 1, stride=1, padding=0),
            nn.BatchNorm2d(out_features),
        )
        self.drop_path = nn.Identity()

    def forward(self, x):
        shortcut = x
        x = self.fc1(x)
        x = self.act(x)
        x = self.fc2(x)
        x = self.drop_path(x) + shortcut
        return x#.reshape(B, C, N, 1)

additional_vig_layers = nn.ModuleList()
for i in range(2):
    additional_vig_layers += Sequential(
        Grapher(in_channels= 64, kernel_size= 9, dilation= 1, conv= "mr", act= "gelu", norm= "batch", bias= True, stochastic= False, epsilon= 0.2,
                r = 1, n= 196, relative_pos= False),
        FFN(64, 64*4, act= "gelu")
    )

new_backbone = Sequential(
    backbone[0:7],
    additional_vig_layers,
    backbone[7:]    
)

In [7]:
new_backbone = Sequential(
    backbone[0:7],
    additional_vig_layers,
    backbone[7:]    
)

In [8]:
print(new_backbone)

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (1): ModuleList(
    (0): Grapher(
      (fc1): Sequential(
        (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (graph_conv): DyGraphConv2d(
        (gconv): MRConv2d(
          (nn): BasicConv(
            (0): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), groups=4)
            (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      

In [9]:
import torch
image_inp = torch.randn(1,3, 800, 1600)
output = backbone(image_inp)

print(output.shape)

torch.Size([1, 512, 25, 50])


In [10]:
new_out = new_backbone(image_inp)

NotImplementedError: Module [ModuleList] is missing the required "forward" function

In [12]:
import torch
import torchvision.models as models
from gcn_lib import Grapher, act_layer
import torch.nn as nn

class VGG16WithAdditionalLayers(nn.Module):
    def __init__(self, num_additional_layers=2):
        super().__init__()
        # Load VGG-16 backbone and split into two parts
        model_backbone = models.vgg16_bn(weights='DEFAULT')
        self.initial_backbone = nn.Sequential(*model_backbone.features[0:7])
        self.final_backbone = nn.Sequential(*model_backbone.features[7:])

        # Define the additional layers
        self.additional_layers = nn.ModuleList()
        for _ in range(num_additional_layers):
            self.additional_layers.append(
                nn.Sequential(
                    Grapher(
                        in_channels=64, kernel_size=9, dilation=1, conv="mr", act="gelu",
                        norm="batch", bias=True, stochastic=False, epsilon=0.2,
                        r=1, n=196, relative_pos=False
                    ),
                    FFN(64, 64 * 4, act="gelu")
                )
            )

    def forward(self, x):
        # Pass through the initial backbone layers
        x = self.initial_backbone(x)

        # Pass through the additional layers
        for layer in self.additional_layers:
            x = layer(x)

        # Pass through the final backbone layers
        x = self.final_backbone(x)
        return x

class FFN(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act='relu', drop_path=0.0):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Sequential(
            nn.Conv2d(in_features, hidden_features, 1, stride=1, padding=0),
            nn.BatchNorm2d(hidden_features),
        )
        self.act = act_layer(act)
        self.fc2 = nn.Sequential(
            nn.Conv2d(hidden_features, out_features, 1, stride=1, padding=0),
            nn.BatchNorm2d(out_features),
        )
        self.drop_path = nn.Identity()

    def forward(self, x):
        shortcut = x
        x = self.fc1(x)
        x = self.act(x)
        x = self.fc2(x)
        x = self.drop_path(x) + shortcut
        return x

# Create an instance of the model
model = VGG16WithAdditionalLayers(num_additional_layers=2)


In [15]:
print(model)

VGG16WithAdditionalLayers(
  (initial_backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (final_backbone): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, pad